In [1]:
import pprint
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import spotipy
import spotipy.util as util
from functools import reduce
from collections import Counter

## SETUP
# C:\MongoDB\bin\mongod.exe --dbpath=C:\MongoDB\data\db --directoryperdb
pp = pprint.PrettyPrinter()
client = MongoClient('localhost', 27017)
db = client.server

In [2]:
## Authentication
username = 'Ariel Godinho'
scope = 'user-top-read user-library-read playlist-modify-private playlist-read-private user-read-playback-state user-modify-playback-state'
token = util.prompt_for_user_token(
    username,
    scope,
    client_id='9c93bd032a4340b086b31bd30ec8f24c',
    client_secret='b893fd0a51d34e2399effa91c1026de7',
    redirect_uri='http://localhost:8888/tree')

if token:
    #Set token to spotify library
    sp = spotipy.Spotify(auth=token)
    print("Token get successful. token", token)
else:
    print("Can't get token for", username)

Token get successful. token BQCYOQ5W7ZCfIfkTwwKC2NfAVxQwe1JI3GZt7LjkGqMKQbqtvqWIdefl73puGI4GGYyrepr__CX-b8AABUfiWplLJXE2z_a8j14emVaZANZfj_e_0-QqQmCmBPMZWs6S2w8NLsviultDmDfp0UQ-wEFg4LkXoB1opHBMnhGbyo0c7g-Ztca5kdHRBiIrZR3DfCsq3f3xV3_4mZ8QXo81


In [3]:
#Buscar artistas mais escutados
f_user_top_artists = sp.current_user_top_artists(50, 0, 'long_term')
pp.pprint([artist['name'] for artist in f_user_top_artists['items']])

# print(len(f_user_top_artists['items']))
#primeiro artista
# pp.pprint(f_user_top_artists['items'][0])

['Natiruts',
 'The Weeknd',
 'Daft Punk',
 'Years & Years',
 'Jorge & Mateus',
 'Drake',
 'Muse',
 'Darren Korb',
 'Cacife Clandestino',
 'Maneva',
 'Cidade Negra',
 'Djavan',
 'Eminem',
 'Disclosure',
 'Avicii',
 'Kamelot',
 'Calvin Harris',
 'Criolo',
 'Seu Jorge',
 'Os Paralamas Do Sucesso',
 'Red Hot Chili Peppers',
 'Lulu Santos',
 'Galantis',
 'Angra',
 'Major Lazer',
 'Black Sabbath',
 'Cássia Eller',
 'Bob Marley & The Wailers',
 'Projota',
 'David Guetta',
 'Incubus',
 'Symphony X',
 'Gorillaz',
 'Tiësto',
 'Arctic Monkeys',
 'Michael Jackson',
 'Ed Sheeran',
 'Queen',
 'Vicetone',
 'Maroon 5',
 'Nirvana',
 'Justin Timberlake',
 'Emicida',
 'Audioslave',
 'Zedd',
 'Tove Lo',
 'Charlie Brown Jr.',
 'Armin van Buuren',
 'Pink Floyd',
 'Michael Bublé']


In [4]:
#buscar seeds de generos possiveis
genre_seeds = sp.recommendation_genre_seeds()['genres']
genre_seeds.append('bossa')
genre_seeds.append('brazilian')
pp.pprint(genre_seeds)


['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [5]:
#Listar generos desses artistas

#Quebrar em palavras
top_genres = [artist['genres'] for artist in f_user_top_artists['items']]
# pp.pprint(top_genres)
top_genres_2d = reduce(lambda x, y: x+y, top_genres)
# pp.pprint(top_genres_2d)
top_genres_2d_split = [genre.split() for genre in top_genres_2d]
# pp.pprint(top_genres_2d_split)
top_genres_2d_one_word = reduce(lambda x, y: x+y, top_genres_2d_split)
# pp.pprint(top_genres_2d_one_word)


# todos os generos (para avaliar perda de generos)
# counter = Counter(top_genres_2d_one_word)
# common_genre_list = [genre[0] for genre in counter.most_common(5)]
# pp.pprint(counter)

# corrigindo generos relacionados
top_genres_2d_one_word = ['bossanova' if genre == 'bossa' else genre for genre in top_genres_2d_one_word]
top_genres_2d_one_word = ['brazil' if genre == 'brazilian' else genre for genre in top_genres_2d_one_word]


#Filtrar por generos que podem ser seed
filtered_top_genres = [x for x in top_genres_2d_one_word if x in genre_seeds]

#Contar generos que mais repetem
counter = Counter(filtered_top_genres)
pp.pprint(counter)

Counter({'rock': 48,
         'pop': 39,
         'metal': 26,
         'brazil': 20,
         'house': 18,
         'dance': 11,
         'edm': 9,
         'alternative': 8,
         'electro': 7,
         'mpb': 6,
         'reggae': 5,
         'funk': 4,
         'latin': 4,
         'samba': 3,
         'classical': 3,
         'trance': 3,
         'swedish': 3,
         'sertanejo': 2,
         'pagode': 2,
         'bossanova': 2,
         'garage': 2,
         'indie': 2,
         'electronic': 1,
         'piano': 1,
         'afrobeat': 1,
         'grunge': 1,
         'german': 1,
         'techno': 1})


In [6]:
#Pegar top 5 generos e buscar 50 ou 100 recomendacoes
common_genre_list = [genre[0] for genre in counter.most_common(5)]
pp.pprint(common_genre_list)

# Get spotify recommendations from genre seeds
recomendation_list = sp.recommendations(seed_genres=common_genre_list, limit=100)
pp.pprint([track['artists'][0]['name'] + ' - ' + track['name'] for track in recomendation_list['tracks']])
# print(recomendation_list['tracks'][0]['artists'][0])                                                            
    

['rock', 'pop', 'metal', 'brazil', 'house']
['The Doors - L.A. Woman',
 'Tiago Iorc - My Girl',
 'Korn - Word Up!',
 'Paramore - Misery Business',
 'Jess Glynne - Take Me Home',
 'Astrud Gilberto - Agua De Beber',
 'Migos - Bad and Boujee (feat. Lil Uzi Vert)',
 'Klingande - Jubel - Radio Edit',
 'Nause - Made Of - Radio Edit',
 'Stanaj - Romantic - NOTD Remix',
 'New Politics - One of Us',
 'Sage The Gemini - Now and Later',
 'Extreme - More Than Words',
 'Maroon 5 - Girls Like You (feat. Cardi B)',
 'Taylor Swift - Gorgeous',
 'Volbeat - Still Counting',
 'Kiiara - Whippin (feat. Felix Snow)',
 'Robin Schulz - Oh Child',
 'Twenty One Pilots - Heathens',
 'Djavan - Sina',
 'Santana - Smooth (feat. Rob Thomas)',
 'Dance Gavin Dance - Midnight Crusade',
 'Avicii - Hey Brother',
 'Little Joy - Keep Me In Mind',
 'Deniz Koyu - Tung!',
 "Judas Priest - You've Got Another Thing Comin'",
 'Living Colour - Cult of Personality',
 'Dynoro - In My Mind',
 'Eric Turner - Dancing In My Head',
 'Kl

In [7]:
#Avaliacao

#avalia se as musicas recomendadas estao no bd (+1 no rating) e se o artista esta no bd (+1 rating)
dict_music_id_rating = {}
artist_list = []
rating = 0
for i in range(len(recomendation_list['tracks'])):
    rating = 0
    
    track_id = recomendation_list['tracks'][i]['id']
    artist_id = recomendation_list['tracks'][i]['artists'][0]['id']
    
    #musica na base
    track = db.tracks.find_one({'id' : track_id})
    if (track is not None):
        rating = rating + 1
#     print(track)
    
    #artista na base
    artist = db.tracks.find_one({'artists_ids' : artist_id})
    if (artist is not None):
        rating = rating + 1
#     print(artist)
    
    artist_list.append(artist_id)
    dict_music_id_rating[track_id] = rating

#dicionario 'musica id': 'rating'
print(dict_music_id_rating)


{'6DmfWj5kOa1fX8AwN9byOn': 2, '6LMjjcvtlWp1Dtj19yiIvT': 2, '4D05EjPFWFx3BEQNeB7Vct': 1, '6SpLc7EXZIPpy0sVko0aoU': 2, '6oO10y1h2VyxJQfomG4CtH': 1, '6hwrxnBXln6pXeMJc6b6Im': 1, '4Km5HrUvYTaSUfiSGPJeQR': 2, '0D2VYiRlBv43asKUgieZaM': 1, '0BxuyVxyAab1f5xL2yuhbm': 1, '7e6FvCvngX5job1PUYIIIL': 2, '3zSaWBoojpQGDPO2oIL80H': 2, '08WPvDEsHvTFuB9w8tC2OS': 2, '1gVgkQFOKa8Wc1HYsJtPdH': 0, '7fa9MBXhVfQ8P8Df9OEbD8': 1, '1ZY1PqizIl78geGM4xWlEA': 2, '6bDVc03eMOMfvKez5qMAcq': 0, '3QWjljChcOMkRDYSzF33Qr': 2, '77VLYTyrdlHEWcjhjQoXlz': 2, '6i0V12jOa3mr6uu4WYhUBr': 2, '2kgT6sMwXd3mdeXhBbLMQe': 2, '0n2SEXB2qoRQg171q7XqeW': 2, '4sykUbPIpLYRfZk1ljZcpj': 0, '3zKST4nk4QJE77oLjUZ0Ng': 1, '5ruFO5Cbjvljsh3ItsDmp8': 2, '1we94xbnyZE1JOIy1fX96F': 1, '4FXv0p8WvZc7eUyDzsi39p': 1, '5e3YOg6fIkP0wD5TyxcHOH': 0, '0E9ZjEAyAwOXZ7wJC0PD33': 2, '6iYQ7v2bpK2kSRZbRfmTRE': 1, '5xrjJANQXCLXeMbNMugyaZ': 1, '4gMgiXfqyzZLMhsksGmbQV': 2, '5o3GnrcFtvkdf3zFznuSbA': 1, '6pbNj2GFxGkBAnpg426pMT': 1, '0nqIqABZcujg05mkixJV60': 1, '0CzeAbfKFnxn

In [8]:
#busca os dados dos artista no spotify 

#Para ser mais eficiente, poderia ser buscado no bd e se nao existisse buscado no spotify
#teria que criar um documento no bd do tipo artista e não apenas referencia-lo no documento track

artists_info_1 = sp.artists(artist_list[0:50])
artists_info_2 = sp.artists(artist_list[50:100])
artists_info = artists_info_1['artists'] + artists_info_2['artists']

In [9]:
#Finalizando a avaliacao

#Cada genero do artista no tops generos do usuario (+1 no rating)

dict_ratings = {}
artist_id_genre_count = {} # dict de id_artista e quantidade de top generos do usuario
for i in range(len(artists_info)):
    
    artist_id = artists_info[i]['id']
    artist_genres = artists_info[i]['genres']
#     print(artist_genres)
    
    if len(artist_genres) != 0:
        artist_genres_2d_split = [genre.split() for genre in artist_genres]
#         print(artist_genres_2d_split)
        artist_genres_2d_one_word = reduce(lambda x, y: x+y, artist_genres_2d_split)
#         print(artist_genres_2d_one_word)
        artist_genres_2d_one_word = ['bossanova' if genre == 'bossa' else genre for genre in artist_genres_2d_one_word]
        artist_genres_2d_one_word = ['brazil' if genre == 'brazilian' else genre for genre in artist_genres_2d_one_word]
#         print(artist_genres_2d_one_word)
    
    filtered_by_user_top_genres = [x for x in artist_genres_2d_one_word if x in common_genre_list]
    filtered_by_user_top_genres = list(set(filtered_by_user_top_genres))
#     print(filtered_by_user_top_genres)
#     print(len(filtered_by_user_top_genres))
    
    artist_id_genre_count[artist_id] = len(filtered_by_user_top_genres)

# verificar se o dict de idArtista + qtd Genero e dict idMusica + rating estao ok
# print(artist_id_genre_count)    
# print(dict_music_id_rating)

for i in range(len(recomendation_list['tracks'])):
    
    track_id = recomendation_list['tracks'][i]['id']
    artist_id = recomendation_list['tracks'][i]['artists'][0]['id']

    #somando ratings para ter o rating total
    dict_ratings[track_id] = dict_music_id_rating[track_id] + artist_id_genre_count[artist_id]
    
#ordernar por maior pontuacao
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
ordenado = sorted(dict_ratings.items(), key=operator.itemgetter(1), reverse=True)

#top 10 musicas recomendadas
print(ordenado[0:10])
    
    

[('1ZozJfi8u9cO2Ob8KwiwNT', 5), ('3oVI4L3rUkawtbRo2GsOZh', 5), ('5sG3G54H21hNfd5etlheoe', 5), ('6LMjjcvtlWp1Dtj19yiIvT', 4), ('3zSaWBoojpQGDPO2oIL80H', 4), ('77VLYTyrdlHEWcjhjQoXlz', 4), ('6i0V12jOa3mr6uu4WYhUBr', 4), ('2kgT6sMwXd3mdeXhBbLMQe', 4), ('60wwxj6Dd9NJlirf84wr2c', 4), ('7dbjRLY969SvR0inx3h1eP', 4)]
